# 0) Prepare base dataframes
    - It cotains import tools, fix seed, load datasets, feature selection for only using interesting variables, etc.. 

    - eICU는 MIMIC과 달리 `offset`이 분 단위로 기록이 되어 있어 시간 관련 변수를 처리할 때 초 단위로 굳이 바꾸지 않고 분 단위를 그대로 활용

    - eICU의 lab 데이터는 자체에 `patientunitstayid` = `stay_id`가 존재하여 따로 stay_id를 할당해주는 작업 불필요

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import random 
import os 
import gc
import math
import json
import optuna
import re
import warnings

from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from datetime import timedelta

from tqdm import tqdm

warnings.simplefilter(action='ignore', category=FutureWarning)

c:\Users\korea\miniconda3\envs\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
def fix_seed(seed: int = 42):
    random.seed(seed) # random
    np.random.seed(seed) # numpy
    os.environ["PYTHONHASHSEED"] = str(seed) # os
    
    # pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed) 
    torch.backends.cudnn.deterministic = True 
    torch.backends.cudnn.benchmark = False 

my_seed = 42
fix_seed(my_seed)

In [34]:
with open('./utils/concept-dict.json', 'r') as f:
    concept_dict = json.load(f)

static_vars = ["age", "sex", "height", "weight"]

dynamic_vars = ["alb", "alp", "alt", "ast", "be", "bicar", "bili", "bili_dir",
                  "bnd", "bun", "ca", "cai", "ck", "ckmb", "cl", "crea", "crp", 
                  "dbp", "fgn", "fio2", "glu", "hgb", "hr", "inr_pt", "k", "lact",
                  "lymph", "map", "mch", "mchc", "mcv", "methb", "mg", "na", "neut", 
                  "o2sat", "pco2", "ph", "phos", "plt", "po2", "ptt", "resp", "sbp", 
                  "temp", "tnt", "urine", "wbc"]

# concept_dict는 다음과 같은 구조로 되어있음 

"""

변수 이름 > unit, min, max, category

-- source 정보 
ids : 해당 변수 고유 식별 번호
table : 어떤 테이블에 존재하는지?
sub_var : 식별 번호가 table의 어떤 column에서 조회해야 하는지?
call_back : 어떠한 전처리를 고려했는지? (e.g.) 단위 변환등 어떤 전처리를 수행해야 하는지 서술해놓음
"""
concept_dict[dynamic_vars[-2]]['sources']

{'aumc': [{'ids': 8794, 'table': 'numericitems', 'sub_var': 'itemid'}],
 'eicu': [{'ids': ['Urine', 'URINE CATHETER'],
   'table': 'intakeoutput',
   'sub_var': 'celllabel'},
  {'regex': 'catheter.+output|output.+catheter',
   'table': 'intakeoutput',
   'sub_var': 'celllabel',
   'class': 'rgx_itm'}],
 'eicu_demo': [{'ids': ['Urine', 'URINE CATHETER'],
   'table': 'intakeoutput',
   'sub_var': 'celllabel'},
  {'regex': 'catheter.+output|output.+catheter',
   'table': 'intakeoutput',
   'sub_var': 'celllabel',
   'class': 'rgx_itm'}],
 'hirid': [{'ids': 30005110,
   'table': 'observations',
   'sub_var': 'variableid',
   'callback': 'hirid_urine',
   'class': 'hrd_itm'}],
 'miiv': [{'ids': [226557,
    226558,
    226559,
    226560,
    226561,
    226563,
    226564,
    226565,
    226566,
    226567,
    226584,
    227510],
   'table': 'outputevents',
   'sub_var': 'itemid'}],
 'mimic': [{'ids': [40055,
    40056,
    40057,
    40065,
    40069,
    40085,
    40086,
    40094,
 

In [35]:
# 해당 데이터셋마다 필요한 값들을 excel 형식으로 변환해보자.

def select_vars(db_name, var_list, base_dict):
    """
    min, max => 이상치 처리 시 사용
    cateogries => 해당 변수 category 분류 시 사용
    source_table => 어느 테이블에서 가져와야 하는지
    source_columns => 해당 테이블 어느 컬럼에서 가져와야 하는지
    source_itemid => 해당 테이블의 컬럼에서 어떤 id로 존재하는지
    """
    var_normal_min = []
    var_normal_max = []
    var_unit = []
    source_category = []
    source_table = []
    source_column = []
    source_itemid = []
    source_callback = []
    source_regex = []
    source_vars = [] # to map other lists
    for var in tqdm(var_list):

        for i in range(len(base_dict[var]['sources'][db_name])): # 여러 테이블에 분산되어 있는 경우 길이가 2가 넘을 수 있기 때문

            source_table.append(base_dict[var]['sources'][db_name][i]['table'])

            try:
                source_column.append(base_dict[var]['sources'][db_name][i]['sub_var'])
            except:
                source_column.append(base_dict[var]['sources'][db_name][i]['val_var'])

            try:
                source_itemid.append(base_dict[var]['sources'][db_name][i]['ids'])
            except:
                source_itemid.append(None)

            try:
                source_callback.append(base_dict[var]['sources'][db_name][i]['callback'])
            except:
                source_callback.append(None)
            
            try:
                source_regex.append(base_dict[var]['sources'][db_name][i]['regex'])
            except:
                source_regex.append(None)

            source_category.append(base_dict[var]['category'])

            try:
                var_unit.append(base_dict[var]['unit'])
            except:
                var_unit.append(None)

            try:
                var_normal_min.append(base_dict[var]['min'])
            except:
                var_normal_min.append(None)
            
            try:
                var_normal_max.append(base_dict[var]['max'])
            except:
                var_normal_max.append(None)


            source_vars.append(var)
        
    return var_normal_min, var_normal_max, var_unit, source_category, source_table, source_column, source_itemid, source_callback, source_vars, source_regex

var_normal_min, var_normal_max, var_unit, source_category, source_table, source_column, source_itemid, source_callback, source_vars, source_regex  = select_vars('hirid', static_vars + dynamic_vars, concept_dict)

MAPPING_DF = pd.DataFrame({
    'var_name' : source_vars,
    'normal_min' : var_normal_min,
    'normal_max' : var_normal_max,
    'category' : source_category,
    'table' : source_table,
    'column' : source_column,
    'itemid' : source_itemid,
    'unit' : var_unit,
    'method' : source_callback,
    'regex' : source_regex
})

urine_rate_row = [{'var_name' : 'urinerate', 'normal_min':None, 'normal_max':None, 'category': 'output', 'table' : 'observations',
                  'column' : 'variableid', 'itemid' : 10020000, 'unit' : 'ml/h', 'method' : None, 'regex' : None}]

# urine rate까지 추가해 놓음
MAPPING_DF = pd.concat([MAPPING_DF, pd.DataFrame(urine_rate_row)], axis = 0).reset_index(drop=True)
MAPPING_DF

100%|██████████| 52/52 [00:00<?, ?it/s]


,var_name,normal_min,normal_max,category,table,column,itemid,unit,method,regex
0,age,0.0,100.0,demographics,general,age,None,years,None,None
1,sex,NaN,NaN,demographics,general,sex,None,None,"apply_map(c(M = 'Male', F = 'Female'))",None
2,height,10.0,230.0,demographics,observations,variableid,10000450,cm,None,None
3,weight,1.0,500.0,demographics,observations,variableid,10000400,kg,None,None
4,alb,0.0,6.0,chemistry,observations,variableid,24000605,g/dL,"convert_unit(binary_op(`*`, 0.1), 'g/dL')",None
5,alp,0.0,NaN,chemistry,observations,variableid,20002700,"[IU/L, U/l]",None,None
6,alt,0.0,NaN,chemistry,observations,variableid,20002600,"[IU/L, U/l]",None,None
7,ast,0.0,NaN,chemistry,observations,variableid,24000330,"[IU/L, U/l]",None,None
8,be,-25.0,25.0,blood gas,observations,variableid,20001300,"[mEq/L, mmol/l]",None,None
9,bicar,5.0,50.0,chemistry,observations,variableid,20004200,"[mEq/L, mmol/l]",None,None


## Data Load & Select only used variables in dataframe

In [5]:
# Original Data Load..
ROOT_DIR = '/Users/korea/datasets/physionet.org/files/hirid/1.1.1'
ROOT_DIR = Path(ROOT_DIR)

# Make Output Folder
if not os.path.exists(ROOT_DIR/'preprocessed_yaib'):
	os.makedirs(ROOT_DIR/'preprocessed_yaib')

# 사실상 observation과 general만 사용하면 됨.
icustays_origin = pd.read_csv(ROOT_DIR/'reference_data'/'general_table.csv') # 이 정보가 patients 즉, icustays와 동일, patientid가 곧 patientid와 동일함
d_items_origin = pd.read_csv(ROOT_DIR/'reference_data'/'hirid_variable_reference.csv')
ordinal_vars = pd.read_csv(ROOT_DIR/'reference_data'/'ordinal_vars_ref.csv')

d_items_origin = d_items_origin[d_items_origin['Source Table'] == 'Observation']

# 매핑 규칙 
"""
1. table > column > itemid 순서대로 매칭 시작 
2. 만약 column이 itemid가 아니면 해당 column 자체의 값을 활용 
3. method가 none이 아니면 단위 변환이 필요
"""

# 사용할 ITEMID LIST 정리
ITEMID_LIST = []
for iid in MAPPING_DF.itemid:
    if iid:
        if not isinstance(iid, list):
            iid = [iid]

        ITEMID_LIST += iid

print(d_items_origin[d_items_origin['ID'].isin(ITEMID_LIST)])

def limit_max_time(df, general_df, max_days):

	res = pd.merge(df, general_df[['patientid', 'admissiontime']], how='left', on='patientid')
	delta = res["datetime"] - res["admissiontime"]
	mask = delta <= pd.Timedelta(days=max_days)

	return res[mask].reset_index(drop=True)


icustays_origin['admissiontime'] = pd.to_datetime(icustays_origin['admissiontime'], format="%Y-%m-%d %H:%M:%S")

icustays_origin["last_obs_time"] = pd.NaT
icustays_origin["los_hours"] = None
icustays_origin["los"] = None

# observation files
for i in tqdm(range(250)):

	if i == 0:
		observation_df = pd.read_parquet(ROOT_DIR/'raw_stage'/'observation_tables_parquet'/'parquet'/f'part-{i}.parquet', engine='fastparquet')

		# ITEMID LIST에 있는 것들로만 사용.
		observation_df = observation_df[observation_df['variableid'].isin(ITEMID_LIST)]

		# 각 itemid마다 어떤 var_name으로 가야하는지 매핑
		observation_df = pd.merge(observation_df, d_items_origin, how = 'left', left_on='variableid', right_on='ID').reset_index(drop=True).drop(columns='ID')
		observation_df = pd.merge(observation_df, MAPPING_DF.explode('itemid', ignore_index=True)[['var_name', 'itemid', 'unit', 'method', 'normal_min', 'normal_max']], how = 'left', left_on = 'variableid', right_on='itemid')
		
		## los 계산
		cur_last = observation_df.groupby('patientid')['datetime'].max().rename('cur_last_obs_time').reset_index()

		icustays_origin = icustays_origin.merge(cur_last, on="patientid", how="left")

		# 누적 max 업데이트
		icustays_origin["last_obs_time"] = icustays_origin[["last_obs_time", "cur_last_obs_time"]].max(axis=1)

		# 임시 컬럼 제거 (중요: 반복 시 컬럼 폭발 방지)
		icustays_origin = icustays_origin.drop(columns=["cur_last_obs_time"])

		mask = (icustays_origin["last_obs_time"].notna()) & (icustays_origin["admissiontime"].notna())
		
		icustays_origin.loc[mask, "los_hours"] = ((icustays_origin.loc[mask, "last_obs_time"] - icustays_origin.loc[mask, "admissiontime"]).dt.total_seconds() / 3600)
		icustays_origin.loc[mask, "los"] = icustays_origin.loc[mask, "los_hours"] / 24

		# 메모리 부족 이슈로 최대 2일까지만 제한
		observation_df = limit_max_time(observation_df, icustays_origin, max_days=2)

		gc.collect()

	else:
		now_df = pd.read_parquet(ROOT_DIR/'raw_stage'/'observation_tables_parquet'/'parquet'/f'part-{i}.parquet', engine='fastparquet')

		# ITEMID LIST에 있는 것들로만 사용.
		now_df = now_df[now_df['variableid'].isin(ITEMID_LIST)]

		# 각 itemid마다 어떤 var_name으로 가야하는지 매핑
		now_df = pd.merge(now_df, d_items_origin, how = 'left', left_on='variableid', right_on='ID').reset_index(drop=True).drop(columns='ID')
		now_df = pd.merge(now_df, MAPPING_DF.explode('itemid', ignore_index=True)[['var_name', 'itemid', 'unit', 'method', 'normal_min', 'normal_max']], how = 'left', left_on = 'variableid', right_on='itemid')
		
		## los계산
		cur_last = now_df.groupby('patientid')['datetime'].max().rename('cur_last_obs_time').reset_index()

		icustays_origin = icustays_origin.merge(cur_last, on="patientid", how="left")

		# 누적 max 업데이트
		icustays_origin["last_obs_time"] = icustays_origin[["last_obs_time", "cur_last_obs_time"]].max(axis=1)

		# 임시 컬럼 제거 (중요: 반복 시 컬럼 폭발 방지)
		icustays_origin = icustays_origin.drop(columns=["cur_last_obs_time"])

		mask = (icustays_origin["last_obs_time"].notna()) & (icustays_origin["admissiontime"].notna())
		
		icustays_origin.loc[mask, "los_hours"] = ((icustays_origin.loc[mask, "last_obs_time"] - icustays_origin.loc[mask, "admissiontime"]).dt.total_seconds() / 3600)
		icustays_origin.loc[mask, "los"] = icustays_origin.loc[mask, "los_hours"] / 24

		# 메모리 부족 이슈로 최대 2일까지만 제한
		now_df = limit_max_time(now_df, icustays_origin, max_days=2)

		observation_df = pd.concat([observation_df, now_df], axis = 0)

		del now_df
		gc.collect()

# 인덱스 초기화
observation_df.reset_index(drop=True, inplace=True)

    Source Table        ID                        Variable Name  Unit  \
0    Observation       200                           Heart rate  /min   
1    Observation       410                Core body temperature    °C   
2    Observation      7100                   Rectal temperature    °C   
3    Observation       400                 Axillary temperature    °C   
4    Observation       100  Invasive systolic arterial pressure  mmHg   
..           ...       ...                                  ...   ...   
132  Observation  20000700       Leukocytes [#/volume] in Blood   G/l   
133  Observation  20000110        Platelets [#/volume] in Blood   G/l   
134  Observation  24000160                   MCH [Entitic mass]    pg   
135  Observation  24000170     MCHC [Mass/volume] in Cord blood   g/L   
136  Observation  24000150                 MCV [Entitic volume]    fl   

    Additional information  
0                      NaN  
1                      NaN  
2                      NaN  
3      

100%|██████████| 250/250 [21:48<00:00,  5.23s/it]


## Rename patientunitstayid columns

In [6]:
# patientid -> stay_id
observation_df.rename(columns={'patientid' : 'stay_id'}, inplace=True)
icustays_origin.rename(columns={'patientid':'stay_id'}, inplace=True)

## Exclusion Criteria & Preprocess Patient(admissionweight, admissionheight) Outlier  

In [7]:
# Exclusion Criteria
def exclusion_criteria(df, min_los, min_age, max_age):
    
    # 1) Invalid LoS -> Maybe Zero
    df = df[df['los']>= 0]
    
    # 2) Min Los
    df = df[df['los']>= min_los]
    
    # 3) Min age : 18
    df = df[(df['age'] >= min_age) & df['age'] <= max_age]
    
    return df.reset_index(drop = True)

icustays_df = icustays_origin.copy()
icustays_df = icustays_df[icustays_df['discharge_status'].notna()]

ex_icustays_df = exclusion_criteria(icustays_df, 1.25, 18, 89).sort_values(['stay_id', 'admissiontime'])
print(f'{icustays_df.shape[0] - ex_icustays_df.shape[0]}명의 환자 삭제. 최종 stay_id 개수 {ex_icustays_df.stay_id.nunique()} 명')

20557명의 환자 삭제. 최종 stay_id 개수 13109 명


# 1) Exclusion Criteria & Checkout Valid Lab events & Outlier & Add time_since_ICU columns

## 1-1) Apply Exclusion Criteria

In [8]:
# def apply_exclusion_criteria(df, ex_icustays_df, time_col_name : str = 'datetime'):
#     """
#     Exclusion Criteria에 맞춰 invalid한 데이터 제거.
#     """
    
#     ## 먼저 intime, outtime의 type을 object -> datetime으로 변경.
#     df[time_col_name] = pd.to_datetime(df[time_col_name], format="%Y-%m-%d %H:%M:%S")
#     ex_icustays_df['admissiontime'] = pd.to_datetime(ex_icustays_df['admissiontime'], format="%Y-%m-%d %H:%M:%S")
#     ex_icustays_df['last_obs_time'] = pd.to_datetime(ex_icustays_df['last_obs_time'], format="%Y-%m-%d %H:%M:%S")
    
#     ## icustays에서 사용할 column만 추리기
#     standard_merge_cols = ['stay_id']
#     standard_icu_cols = ['stay_id','admissiontime','last_obs_time']
    
#     merged_df = pd.merge(df, ex_icustays_df[standard_icu_cols], how = 'inner', on = standard_merge_cols) # inner로 겹치는 것들만 추리기.

        
#     merged_df = merged_df[(merged_df['admissiontime'] <= merged_df[time_col_name]) & (merged_df[time_col_name] <= merged_df['last_obs_time'])] # charttime이 intime과 outtime 사이에 있는 값만 사용. 이외 제외.
        
        
#     print(f'Result {df.shape[0]} ---> {merged_df.shape[0]}, {df.shape[0] - merged_df.shape[0]} Delete!')
        
#     return merged_df.drop(columns = ['admissiontime', 'last_obs_time'])

# # Delete exclusion criterion stay_id in dataframe.
# observation_df = apply_exclusion_criteria(observation_df, ex_icustays_df, 'datetime')

observation_df = observation_df[observation_df['stay_id'].isin(ex_icustays_df['stay_id'].unique())].reset_index(drop=True)

## 1-2) Unit Conversion

In [9]:
# 단위 변경해야할 것은 alb, bili, bili_dir, bun, ca, crea, fgn, glu, hgb, lymph, mchc, mg, phos, tnt, urine
display(MAPPING_DF[MAPPING_DF['method'].notna()])

checklist = pd.merge(observation_df[['var_name', 'itemid', 'Unit']].drop_duplicates().sort_values('var_name'), MAPPING_DF[MAPPING_DF['table'] == 'observations'][['var_name', 'unit']].sort_values('var_name'), on='var_name')

check_vars = []
for _, row in checklist.iterrows():
    if row['Unit'] not in [row['unit']]:
        check_vars.append(row['var_name'])

display(checklist[checklist['var_name'].isin(check_vars)])

,var_name,normal_min,normal_max,category,table,column,itemid,unit,method,regex
1,sex,NaN,NaN,demographics,general,sex,None,None,"apply_map(c(M = 'Male', F = 'Female'))",None
4,alb,0.0,6.0,chemistry,observations,variableid,24000605,g/dL,"convert_unit(binary_op(`*`, 0.1), 'g/dL')",None
10,bili,0.0,100.0,chemistry,observations,variableid,20004300,mg/dL,"convert_unit(binary_op(`*`, 0.058467), 'mg/dL')",None
11,bili_dir,0.0,50.0,chemistry,observations,variableid,24000560,mg/dL,"convert_unit(binary_op(`*`, 0.058467), 'mg/dL')",None
13,bun,0.0,200.0,chemistry,observations,variableid,20004100,mg/dL,"convert_unit(binary_op(`*`, 2.8), 'mg/dL')",None
14,ca,4.0,20.0,chemistry,observations,variableid,20005100,mg/dL,"convert_unit(binary_op(`*`, 4.008), 'mg/dL')",None
19,crea,0.0,15.0,chemistry,observations,variableid,20000600,mg/dL,"convert_unit(binary_op(`*`, 0.011312), 'mg/dL')",None
22,fgn,0.0,1500.0,hematology,observations,variableid,24000536,mg/dL,"convert_unit(binary_op(`*`, 100), 'mg/dL')",None
24,glu,0.0,1000.0,chemistry,observations,variableid,"[20005110, 24000523, 24000585]",mg/dL,"convert_unit(binary_op(`*`, 18.016), 'mg/dL')",None
25,hgb,4.0,18.0,hematology,observations,variableid,"[24000548, 24000836, 20000900]",g/dL,"convert_unit(binary_op(`*`, 0.1), 'g/dL', 'g/l')",None


,var_name,itemid,Unit,unit
0,alb,24000605,g/L,g/dL
1,alp,20002700,U/l,"[IU/L, U/l]"
2,alt,20002600,U/l,"[IU/L, U/l]"
3,ast,24000330,U/l,"[IU/L, U/l]"
4,be,20001300,mmol/l,"[mEq/L, mmol/l]"
5,bicar,20004200,mmol/l,"[mEq/L, mmol/l]"
6,bili,20004300,umol/l,mg/dL
7,bili_dir,24000560,umol/l,mg/dL
9,bun,20004100,mmol/l,mg/dL
10,ca,20005100,mmol/l,mg/dL


In [10]:
display(MAPPING_DF[MAPPING_DF['method'].notna()])
display(checklist[checklist['var_name'].isin(MAPPING_DF[MAPPING_DF['method'].notna()].var_name.values)])

,var_name,normal_min,normal_max,category,table,column,itemid,unit,method,regex
1,sex,NaN,NaN,demographics,general,sex,None,None,"apply_map(c(M = 'Male', F = 'Female'))",None
4,alb,0.0,6.0,chemistry,observations,variableid,24000605,g/dL,"convert_unit(binary_op(`*`, 0.1), 'g/dL')",None
10,bili,0.0,100.0,chemistry,observations,variableid,20004300,mg/dL,"convert_unit(binary_op(`*`, 0.058467), 'mg/dL')",None
11,bili_dir,0.0,50.0,chemistry,observations,variableid,24000560,mg/dL,"convert_unit(binary_op(`*`, 0.058467), 'mg/dL')",None
13,bun,0.0,200.0,chemistry,observations,variableid,20004100,mg/dL,"convert_unit(binary_op(`*`, 2.8), 'mg/dL')",None
14,ca,4.0,20.0,chemistry,observations,variableid,20005100,mg/dL,"convert_unit(binary_op(`*`, 4.008), 'mg/dL')",None
19,crea,0.0,15.0,chemistry,observations,variableid,20000600,mg/dL,"convert_unit(binary_op(`*`, 0.011312), 'mg/dL')",None
22,fgn,0.0,1500.0,hematology,observations,variableid,24000536,mg/dL,"convert_unit(binary_op(`*`, 100), 'mg/dL')",None
24,glu,0.0,1000.0,chemistry,observations,variableid,"[20005110, 24000523, 24000585]",mg/dL,"convert_unit(binary_op(`*`, 18.016), 'mg/dL')",None
25,hgb,4.0,18.0,hematology,observations,variableid,"[24000548, 24000836, 20000900]",g/dL,"convert_unit(binary_op(`*`, 0.1), 'g/dL', 'g/l')",None


,var_name,itemid,Unit,unit
0,alb,24000605,g/L,g/dL
6,bili,20004300,umol/l,mg/dL
7,bili_dir,24000560,umol/l,mg/dL
9,bun,20004100,mmol/l,mg/dL
10,ca,20005100,mmol/l,mg/dL
16,crea,20000600,umol/l,mg/dL
19,fgn,24000536,g/L,mg/dL
21,glu,24000585,mmol/l,mg/dL
22,glu,20005110,mmol/l,mg/dL
23,glu,24000523,mmol/l,mg/dL


In [11]:
# 전처리 하기에 앞서 먼저 중복된 값들은 제거 및 entertime이 가장 최신인 것들로 유지하도록 변경.

# 1. 기준 컬럼들로 먼저 정렬 (예: stay_id별로 시간순 정렬)
observation_df = observation_df.sort_values(by=["stay_id", "datetime", "entertime","var_name"])
print(f'Before drop duplicates {observation_df.shape[0]}')

# 2. 마지막 행(가장 나중 시간)만 남기고 중복 제거
observation_df = observation_df.drop_duplicates(subset=["stay_id", "datetime", "var_name"], keep="last").reset_index(drop=True)
print(f'After drop duplicates {observation_df.shape[0]}')

Before drop duplicates 118087370
After drop duplicates 114569357


In [12]:
# 단위 변경해야할 것은 alb, bili, bili_dir, bun, ca, crea, fgn, glu, hgb, lymph, mchc, mg, phos, tnt, urine
# lymph는 같은 시점의 WBC 값을 기준으로 나눠준 뒤 100을 곱해주어야 함.
def alb_gl_to_gdl(x):
    return x * 0.1

def bili_umol_to_mgdl(x):
    return x * 0.058467

def bili_dir_umol_to_mgdl(x):
    return x * 0.058467

def bun_mmol_to_mgdl(x):
    return x * 2.8

def ca_mmol_to_mgdl(x):
    return x * 4.008

def crea_umol_to_mgdl(x):
    return x * 0.011312

def fgn_gl_to_mgdl(x):
    return x * 100

def glu_mmol_to_mgdl(x):
    return x * 18.016

def hgb_gl_to_gdl(x):
    return x * 0.1

def mchc_gl_to_per(x):
    return x * 0.1

def mg_mmol_to_mgdl(x):
    return x * 2.431

def phos_mmol_to_mgdl(x):
    return x * 3.097521

def tnt_ngl_to_ngml(x):
    return x / 1000


observation_df.loc[observation_df['var_name']=='alb', 'value'] = observation_df.loc[observation_df['var_name']=='alb', 'value'].apply(alb_gl_to_gdl)
observation_df.loc[observation_df['var_name']=='bili', 'value'] = observation_df.loc[observation_df['var_name']=='bili', 'value'].apply(bili_umol_to_mgdl)
observation_df.loc[observation_df['var_name']=='bili_dir', 'value'] = observation_df.loc[observation_df['var_name']=='bili_dir', 'value'].apply(bili_dir_umol_to_mgdl)
observation_df.loc[observation_df['var_name']=='bun', 'value'] = observation_df.loc[observation_df['var_name']=='bun', 'value'].apply(bun_mmol_to_mgdl)
observation_df.loc[observation_df['var_name']=='ca', 'value'] = observation_df.loc[observation_df['var_name']=='ca', 'value'].apply(ca_mmol_to_mgdl)
observation_df.loc[observation_df['var_name']=='crea', 'value'] = observation_df.loc[observation_df['var_name']=='crea', 'value'].apply(crea_umol_to_mgdl)
observation_df.loc[observation_df['var_name']=='fgn', 'value'] = observation_df.loc[observation_df['var_name']=='fgn', 'value'].apply(fgn_gl_to_mgdl)
observation_df.loc[observation_df['var_name']=='glu', 'value'] = observation_df.loc[observation_df['var_name']=='glu', 'value'].apply(glu_mmol_to_mgdl)
observation_df.loc[observation_df['var_name']=='hgb', 'value'] = observation_df.loc[observation_df['var_name']=='hgb', 'value'].apply(hgb_gl_to_gdl)
observation_df.loc[observation_df['var_name']=='mchc', 'value'] = observation_df.loc[observation_df['var_name']=='mchc', 'value'].apply(mchc_gl_to_per)
observation_df.loc[observation_df['var_name']=='mg', 'value'] = observation_df.loc[observation_df['var_name']=='mg', 'value'].apply(mg_mmol_to_mgdl)
observation_df.loc[observation_df['var_name']=='phos', 'value'] = observation_df.loc[observation_df['var_name']=='phos', 'value'].apply(phos_mmol_to_mgdl)
observation_df.loc[observation_df['var_name']=='tnt', 'value'] = observation_df.loc[observation_df['var_name']=='tnt', 'value'].apply(tnt_ngl_to_ngml)

In [13]:
# urine output은 누적값이므로 이전 기록 시점과의 차분된 값으로 사용
def cumulative_to_event(df, stay_col="stay_id", time_col="datetime", value_col="value"):
    res = df.copy()
    res[time_col] = pd.to_datetime(res[time_col], errors="coerce")

    res = res.sort_values([stay_col, time_col])
    res["event_ml"] = res.groupby(stay_col)[value_col].diff()

    # reset/정정으로 누적이 줄어든 경우(음수 diff)는 0으로 처리
    res["event_ml"] = res["event_ml"].clip(lower=0)

    return res.dropna(subset='event_ml').reset_index(drop=True) # 이전 기록이 없는 맨 처음 값인 경우 삭제함.

urine_df = cumulative_to_event(observation_df[observation_df['var_name']=='urine'], 'stay_id', 'datetime', 'value')

# 후처리
urine_df['value'] = urine_df['event_ml']
urine_df.drop(columns='event_ml', inplace = True)
display(urine_df)

,datetime,entertime,stay_id,status,stringvalue,type,value,variableid,Source Table,Variable Name,Unit,Additional information,var_name,itemid,unit,method,normal_min,normal_max,admissiontime
0,2117-10-19 05:34:00,2117-10-19 05:34:32.430,2,8,,,304.030670,30005110,Observation,Fluid balance,cummulative,NaN,urine,30005110,mL,hirid_urine,0.0,2000.0,2117-10-18 22:35:00
1,2117-10-19 11:00:00,2117-10-19 12:03:10.230,2,8,,,371.962402,30005110,Observation,Fluid balance,cummulative,NaN,urine,30005110,mL,hirid_urine,0.0,2000.0,2117-10-18 22:35:00
2,2117-10-19 11:59:59,2117-10-19 12:09:56.100,2,8,,,38.361328,30005110,Observation,Fluid balance,cummulative,NaN,urine,30005110,mL,hirid_urine,0.0,2000.0,2117-10-18 22:35:00
3,2117-10-19 12:00:00,2117-10-19 15:21:47.360,2,8,,,0.000000,30005110,Observation,Fluid balance,cummulative,NaN,urine,30005110,mL,hirid_urine,0.0,2000.0,2117-10-18 22:35:00
4,2117-10-19 15:22:00,2117-10-19 15:21:47.360,2,8,,,571.338684,30005110,Observation,Fluid balance,cummulative,NaN,urine,30005110,mL,hirid_urine,0.0,2000.0,2117-10-18 22:35:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427590,2162-03-21 20:42:00,2162-03-21 20:42:16.250,33904,8,,,366.680908,30005110,Observation,Fluid balance,cummulative,NaN,urine,30005110,mL,hirid_urine,0.0,2000.0,2162-03-19 23:35:00
427591,2162-03-21 21:15:00,2162-03-21 21:16:02.060,33904,8,,,442.575928,30005110,Observation,Fluid balance,cummulative,NaN,urine,30005110,mL,hirid_urine,0.0,2000.0,2162-03-19 23:35:00
427592,2162-03-21 21:30:00,2162-03-21 21:30:26.040,33904,8,,,210.261963,30005110,Observation,Fluid balance,cummulative,NaN,urine,30005110,mL,hirid_urine,0.0,2000.0,2162-03-19 23:35:00
427593,2162-03-21 22:07:00,2162-03-21 22:10:22.560,33904,8,,,345.312744,30005110,Observation,Fluid balance,cummulative,NaN,urine,30005110,mL,hirid_urine,0.0,2000.0,2162-03-19 23:35:00


In [14]:
# lymph을 % 단위로 변경하기
# lymph는 같은 시점의 WBC 값을 기준으로 나눠준 뒤 100을 곱해주어야 함.
def lymph_gl_to_percent(df, stay="stay_id", t="datetime", var="var_name", val="value"):
    res = df.copy()
    res[t] = pd.to_datetime(res[t], errors="coerce")

    L = res[res[var]=='lymph'][[stay, t, val]].rename(columns={val: "L"}).sort_values([t, stay])
    W = res[res[var]=='wbc'][[stay, t, val]].rename(columns={val: "W"}).sort_values([t, stay])

    # 같은 시점 있으면 그대로, 없으면 이전 시점 중 가장 최근 WBC 매칭하되 최대 24시간 전의 값까지만 활용.
    m = pd.merge_asof(L, W, on=t, by=stay, direction="backward", tolerance=pd.Timedelta("24h"))
    m["pct"] = np.where(m["W"].notna() & (m["W"] != 0), m["L"] / m["W"] * 100.0, np.nan)

    display(L)
    display(W)
    display(m)

    # lymph row만 value(%)로 교체
    key = [stay, t]
    res = res.merge(m[key + ["pct"]], on=key, how="left")
    mask = (res[var]=='lymph') & (res["pct"].notna())
    res.loc[mask, val] = res.loc[mask, "pct"]

    return res[res[var]=='lymph'].drop(columns=["pct"]).reset_index(drop=True)

lymph_df = lymph_gl_to_percent(observation_df[observation_df['var_name'].isin(['lymph', 'wbc'])].sort_values(['stay_id', 'datetime']))
display(lymph_df)

,stay_id,datetime,L
57450438,16910,2102-11-27 08:29:00,0.00
57455796,16910,2102-11-28 06:58:00,0.00
28113652,8240,2103-01-23 09:21:00,0.00
28118691,8240,2103-01-24 06:00:00,0.13
80270491,23745,2103-01-31 01:23:00,0.00
...,...,...,...
22259619,6538,2198-07-11 06:00:00,0.02
30333784,8854,2198-07-14 22:00:00,0.00
30335761,8854,2198-07-15 06:14:00,0.00
30342026,8854,2198-07-16 06:00:00,0.00


,stay_id,datetime,W
57450435,16910,2102-11-27 08:29:00,14.300000
57455793,16910,2102-11-28 06:58:00,8.700000
28112330,8240,2103-01-23 01:20:00,14.100000
28113649,8240,2103-01-23 09:21:00,4.500000
28115164,8240,2103-01-23 16:06:00,4.100000
...,...,...,...
30342017,8854,2198-07-16 06:00:00,26.799999
110057350,32600,2198-08-19 06:00:00,5.800000
101347820,29935,2198-09-03 17:56:00,8.600000
101350624,29935,2198-09-04 06:00:00,8.200000


,stay_id,datetime,L,W,pct
0,16910,2102-11-27 08:29:00,0.00,14.300000,0.000000
1,16910,2102-11-28 06:58:00,0.00,8.700000,0.000000
2,8240,2103-01-23 09:21:00,0.00,4.500000,0.000000
3,8240,2103-01-24 06:00:00,0.13,27.100000,0.479705
4,23745,2103-01-31 01:23:00,0.00,6.700000,0.000000
...,...,...,...,...,...
9471,6538,2198-07-11 06:00:00,0.02,3.900000,0.512820
9472,8854,2198-07-14 22:00:00,0.00,18.799999,0.000000
9473,8854,2198-07-15 06:14:00,0.00,22.700001,0.000000
9474,8854,2198-07-16 06:00:00,0.00,26.799999,0.000000


,datetime,entertime,stay_id,status,stringvalue,type,value,variableid,Source Table,Variable Name,Unit,Additional information,var_name,itemid,unit,method,normal_min,normal_max,admissiontime
0,2149-01-10 04:49:00,2149-01-11 00:30:21.140,4,8,None,F,0.549451,24000480,Observation,Lymphocytes [#/volume] in Blood,G/l,NaN,lymph,24000480,%,blood_cell_ratio,0.0,100.0,2149-01-08 15:30:00
1,2144-06-08 06:00:00,2144-06-08 08:24:42.483,5,8,None,F,0.263158,24000480,Observation,Lymphocytes [#/volume] in Blood,G/l,NaN,lymph,24000480,%,blood_cell_ratio,0.0,100.0,2144-06-06 16:15:00
2,2177-05-07 14:02:00,2177-05-08 00:59:45.110,11,8,None,F,0.000000,24000480,Observation,Lymphocytes [#/volume] in Blood,G/l,NaN,lymph,24000480,%,blood_cell_ratio,0.0,100.0,2177-05-07 13:40:00
3,2116-01-25 00:40:00,2116-01-26 03:04:11.566,24,8,None,F,0.000000,24000480,Observation,Lymphocytes [#/volume] in Blood,G/l,NaN,lymph,24000480,%,blood_cell_ratio,0.0,100.0,2116-01-25 00:10:00
4,2153-11-25 10:19:00,2153-11-26 00:31:49.603,30,8,None,F,0.500000,24000480,Observation,Lymphocytes [#/volume] in Blood,G/l,NaN,lymph,24000480,%,blood_cell_ratio,0.0,100.0,2153-11-25 09:15:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9471,2178-02-16 06:00:00,2178-02-17 04:47:19.186,33892,8,0.00,F,0.000000,24000480,Observation,Lymphocytes [#/volume] in Blood,G/l,NaN,lymph,24000480,%,blood_cell_ratio,0.0,100.0,2178-02-14 15:40:00
9472,2178-02-16 11:30:00,2178-02-17 04:52:56.780,33892,8,0.00,F,0.000000,24000480,Observation,Lymphocytes [#/volume] in Blood,G/l,NaN,lymph,24000480,%,blood_cell_ratio,0.0,100.0,2178-02-14 15:40:00
9473,2119-06-17 06:00:00,2119-06-18 00:21:34.960,33901,8,None,F,0.000000,24000480,Observation,Lymphocytes [#/volume] in Blood,G/l,NaN,lymph,24000480,%,blood_cell_ratio,0.0,100.0,2119-06-15 21:55:00
9474,2162-03-20 00:30:00,2162-03-21 00:21:22.546,33904,8,None,F,0.000000,24000480,Observation,Lymphocytes [#/volume] in Blood,G/l,NaN,lymph,24000480,%,blood_cell_ratio,0.0,100.0,2162-03-19 23:35:00


In [15]:
# 단위 변환된 변수들 최종적으로 결합하기
observation_df = observation_df[~observation_df['var_name'].isin(['urine', 'lymph'])] # urine, lymph만 제외한 것
observation_df = pd.concat([observation_df, urine_df, lymph_df], axis = 0).sort_values(['stay_id', 'datetime']).reset_index(drop=True)

In [16]:
del urine_df, lymph_df
gc.collect()

0

## 1-3) Preprocess Outlier & Split to each purpose dataframe

- Drugs, Ventilator, Weight Dataframe을 제외한 나머지 데이터프레임에 대해서 이상치 처리. (Vital, Lab, UrineOutput, Score)

- 먼저, 각 데이터프레임의 value 값들을 수치형 변수로 바꿔줘야함.

- 동일한 변수더라도 테이블내에 이름이 다르거나 하면 다른 변수로 처리해서 우선 진행.

- 전처리 이후 MIMIC처럼 Chart , Lab 이렇게 통일시키는게 좋을듯

- vital관련 변수들은 melt해서 바꿔서 처리하거나 직접 값을 찾아서 해야할듯

In [17]:
# Outlier Removal by min, max values
# urine_df, respiratoryCharting_df, lab_df, vitalPeriodic_df, vitalAperiodic_df

# def filter_outliers(df, value_col):
#     """
#     연속형 변수 처리하기 위한 함수 
#     """
#     def _filtering(row):
#         if not np.isnan(row['normal_min']) and not np.isnan(row['normal_max']):
#             return row['normal_min'] <= row[value_col] <= row['normal_max'] 

#         elif not np.isnan(row['normal_min']) and np.isnan(row['normal_max']): # min값만 있을 때
#             return row['normal_min'] <= row[value_col]

#         elif np.isnan(row['normal_min']) and not np.isnan(row['normal_max']): # max값만 있을 때 
#             return row[value_col] <= row['normal_max'] 

#         else:
#             return True

#     df_filtered = df.copy()
#     outlier_binary = df.apply(_filtering, axis = 1)
#     df_filtered['out_bin'] = outlier_binary

#     df_filtered = df_filtered[df_filtered['out_bin']].reset_index(drop = True)

#     return df_filtered

# observation_df = filter_outliers(observation_df, 'value')

def filter_outliers_fast(df, value_col="value", min_col="normal_min", max_col="normal_max"):
    v = df[value_col].to_numpy()
    lo = df[min_col].to_numpy()
    hi = df[max_col].to_numpy()

    # min/max가 NaN이면 해당 조건은 True로 처리
    mask = np.ones(len(df), dtype=bool)

    lo_ok = ~np.isnan(lo)
    hi_ok = ~np.isnan(hi)

    mask &= (~lo_ok) | (v >= lo)
    mask &= (~hi_ok) | (v <= hi)

    # out_bin 컬럼 굳이 만들지 않고 바로 필터
    return df.loc[mask].reset_index(drop=True)

observation_df = filter_outliers_fast(observation_df, "value")

gc.collect()

0

In [18]:
observation_df.to_csv(ROOT_DIR/'preprocessed_yaib'/'observation_df.csv.gz', index = False, compression='gzip')

# 2) Disease Annotation

## 2-1) AKI Annotation

- AKI Labeling은 입원 이전 7일 이내의 baseline을 구해야하기 때문에 original labevent dataframe으로 수행. 

### 2-1-1) Preparing Variables

In [19]:
# # labevent에서는 creatinine을 사용
# def assign_stayid_to_lab(lab_event, ex_icustays_df, valid_days = 1):
#     valid_lab = lab_event[lab_event['hadm_id'].notna()].query('hadm_id.isin(@ex_icustays_df.hadm_id.unique())')

#     new_lab_rows = []
    
#     valid_lab['charttime'] = pd.to_datetime(valid_lab['charttime'], format="%Y-%m-%d %H:%M:%S")
#     ex_icustays_df['intime'] = pd.to_datetime(ex_icustays_df['intime'], format="%Y-%m-%d %H:%M:%S")
#     ex_icustays_df['outtime'] = pd.to_datetime(ex_icustays_df['outtime'], format="%Y-%m-%d %H:%M:%S")

#     valid_lab['stay_id'] = np.nan

#     for _, row in tqdm(valid_lab.iterrows(), total = valid_lab.shape[0], desc = 'assining_stay_id'):
#         standard_icu = ex_icustays_df[ex_icustays_df['hadm_id'] == row.hadm_id].sort_values('intime')
            
#         for _, stay in standard_icu.iterrows():
            
#             if stay['intime'] - pd.Timedelta(days=valid_days) <= row['charttime'] <= stay['outtime']: # 몇 일 전 데이터까지 사용할 것인지?
#                 row['stay_id'] = stay['stay_id']
#                 new_lab_rows.append(row)
#                 break
#             else:
#                 continue

#     result = pd.DataFrame(new_lab_rows)
    
#     return result

# creat_df = assign_stayid_to_lab(lab_df[lab_df['itemid'] == 'creatinine'].reset_index(drop=True), ex_icustays_df, 7) # 중환자실 입원 전 7일 이내의 데이터만 사용.

creat_df = observation_df[observation_df['itemid'] == 20000600].reset_index(drop=True)
creat_df['unit'] = creat_df['unit'].astype('string')
creat_df.to_parquet(ROOT_DIR/'preprocessed_yaib'/f'creat_for_labeling_df.parquet', index=False)

# admission weight
adm_weights = observation_df[observation_df['itemid']==10000400].reset_index(drop = True)

# 사용 변수 설정
common_cols = ['stay_id', 'datetime', 'value']

creat_df = creat_df[common_cols].sort_values(['stay_id', 'datetime']).reset_index(drop=True)

creat_df.rename(columns={'value' : 'creatinine'}, inplace = True)
adm_weights.rename(columns={'value' : 'admission_weight'}, inplace = True)


### 2-1-2) Caculate creatinine baseline & urine rate

In [20]:
# Exclusion Cohort용 baseline 계산

# 환자별로 우선 exclusion criteria를 위한 baseline 계산
def compute_baselines_for_excl(stay_df, creat_df):
    """
    YAIB 논문에 따르면 exclusion criteria를 계산하기 위해 입원 전 가장 최근값 or ICU 입원 후 가장 빠른 값이 4mg/dL이 넘으면 제외한다고 함.
    """
    
    def _baseline_preicu_last_or_earliest_in_icu(creat_df, icu_intime):
        # last prior to ICU
        prior = creat_df[creat_df['datetime'] < icu_intime]
        if not prior.empty:
            # 가장 최근값
            return prior.sort_values('datetime', ascending=False).iloc[0]['creatinine']
        
        # 없으면 ICU 내의 earliest
        in_icu = creat_df[creat_df['datetime'] >= icu_intime]

        if not in_icu.empty:
            return in_icu.sort_values('datetime').iloc[0]['creatinine']
        
        return np.nan
    
    rows = []
    for stay in tqdm(stay_df.stay_id.unique()):
        csub = creat_df[creat_df['stay_id'] == stay]
        icu_time = stay_df[stay_df['stay_id'] == stay]['admissiontime'].values[0]
        
        bpre = _baseline_preicu_last_or_earliest_in_icu(csub, icu_time)
        rows.append({'stay_id':stay, 'baseline_preicu_or_earliest': bpre})

    return pd.DataFrame(rows)

# baseline 계산 
baselines_df = compute_baselines_for_excl(ex_icustays_df, creat_df)

# exclusion mask 생성
aki_exclude_mask = baselines_df['baseline_preicu_or_earliest'] > 4.0
aki_excluded_stays = baselines_df.loc[aki_exclude_mask, 'stay_id'].tolist()


100%|██████████| 13109/13109 [00:08<00:00, 1542.83it/s]


In [21]:
# labeling 계산을 위한 baseline 계산

def compute_baseline_7d(group):
    """
    하나의 row마다 이전 7일 이내의 과거 값들 중 최소값을 baseline으로 설정
    """
    # set index -> rolling('7D') will consider previous 7 days up to current index
    g = group.set_index('datetime').sort_index()

    # rolling window of 7 days, exclude current row by closed='left'
    baseline_series = g['creatinine'].rolling('7D', closed='left').min()

    # baseline for earliest points will be NaN if no prior values
    g = g.assign(baseline_7d = baseline_series)

    return g.reset_index()

baseline_7d_df = creat_df.groupby('stay_id', group_keys = False).apply(compute_baseline_7d)
baseline_7d_df = pd.merge(baseline_7d_df, baselines_df, how = 'left', on = 'stay_id')
baseline_7d_df['baseline_7d'] = baseline_7d_df['baseline_7d'].fillna(baseline_7d_df['baseline_preicu_or_earliest']) # 만약 baseline이 결측값이면 icu 입원 전 마지막 or icu 입원 후 첫 번째 값으로 채움

In [22]:
# Baseline 기준으로 flag 계산
tqdm.pandas()

def compute_baseline_rules(group):
    """
    rule 1 : 해당 시점 기준 48시간 이내 0.3mg/dL 이상 증가 여부
    rule 2 : baseline 대비 1.5배 이상 증가 여부
    """
    # group: 하나의 stay_id 그룹
    group = group.copy()
    group.set_index('datetime', inplace=True)
    
    # 이전 row(즉, 현재 시점 제외)의 Creatinine 값을 기반으로 지난 48시간의 최소값을 계산
    group['min_creat_48h'] = group['creatinine'].shift(1).rolling('48h').min()
    group['rule1'] = (group['creatinine'] - group['min_creat_48h'] >= 0.3)
    group['rule1'] = group['rule1'].fillna(False)
    
    # Rule 2: 현재 Creatinine 값이 baseline_cr의 1.5배 이상인가?
    group['rule2'] = (group['creatinine'] >= group['baseline_7d'] * 1.5)
    
    return group.reset_index()

aki_baseline_flag = baseline_7d_df.groupby('stay_id', group_keys=False).progress_apply(compute_baseline_rules)
aki_baseline_flag['AKI_Annotation'] = np.where((aki_baseline_flag['rule1'] | aki_baseline_flag['rule2']),1,0)

100%|██████████| 12497/12497 [00:13<00:00, 929.96it/s] 


In [23]:
# urine rate 계산
# hirid는 이미 urine rate이 있기 때문에 단순히 admissionweight으로 나눠주면 됨.
def cal_urine_rate(outputevent_df, adm_weight_df):

    df = outputevent_df.copy()
    df = df.sort_values(['stay_id','datetime'])
    df['prev_time'] = df.groupby('stay_id')['datetime'].shift(1)

    df['elapsed_hours'] = (df['datetime'] - df['prev_time']).dt.total_seconds() / 3600.0

    # apply rules: earliest -> 1h, cap at 24h, avoid zero
    df['elapsed_hours'] = df['elapsed_hours'].fillna(1.0)            # earliest = 1h
    df['elapsed_hours'] = df['elapsed_hours'].replace(0, 1.0)        # zero gap -> treat as 1h (or small eps)
    df['elapsed_hours'] = df['elapsed_hours'].clip(upper=24.0)       # max gap = 24h

    weight_df = adm_weight_df[['stay_id', 'datetime', 'admission_weight']].copy()

    df = df.sort_values('datetime')
    weight_df = weight_df.sort_values('datetime')

    # merge weight (use 75 kg if missing)
    df = pd.merge_asof(df, weight_df, on = 'datetime', by = 'stay_id', direction='backward')
    df['admission_weight'] = df['admission_weight'].fillna(75.0)

    df['urine_rate'] = df['value'] / df['admission_weight']

    return df

urine_rate_df = cal_urine_rate(outputevent_df=observation_df[observation_df['var_name']=='urinerate'], adm_weight_df=adm_weights)
gc.collect()

30

In [24]:
# 이 urine rate을 기반으로 가중 평균을 취해서 KDIGO 기준 annotation flag를 생성
def add_timeweighted_uo(group, window_hours=6, min_coverage_ratio=0.8):
    g = group.set_index('datetime')

    # 1) interval별 volume 계산 (ml)
    g['volume_ml'] = g['urine_rate'] * g['admission_weight'] * g['elapsed_hours']

    # 2) 지난 6시간 동안 volume 합, 시간 합
    win = f'{window_hours}h'
    vol_roll = g['volume_ml'].rolling(win, min_periods=1)
    dt_roll  = g['elapsed_hours'].rolling(win, min_periods=1)

    g['vol_6h_sum']   = vol_roll.sum()
    g['hours_6h_sum'] = dt_roll.sum()

    # 3) 시간가중 평균 urine rate (mL/kg/h)
    #    UO_rate_6h = 총 volume / (weight * 실제 커버된 시간)
    g['uo_6h_timeweighted'] = (
        g['vol_6h_sum'] / (g['admission_weight'] * g['hours_6h_sum'])
    )

    # 4) KDIGO urine 기준: 6h 평균 < 0.5 mL/kg/h
    g['AKI_Annotation'] = (g['uo_6h_timeweighted'] < 0.5).astype(int)

    return g.reset_index()

aki_urine_flag = (
    urine_rate_df
    .groupby('stay_id', group_keys=False)
    .apply(add_timeweighted_uo)
)

### 2-1-3) AKI Annotation

In [25]:
# AKI Annotation
def add_aki_annotation(baseline_flag, urine_flag, stays):
    """
    사용해야 하는 변수 : weight, urine output, 입원 이전 creatinine(baseline 계산용), 입원 이후 creatinine
    """

    use_cols = ['stay_id', 'datetime', 'AKI_Annotation']
    res_aki = pd.concat([baseline_flag[use_cols], urine_flag[use_cols]], axis = 0)

    print(f'Drop Duplicates : {res_aki.shape[0]} --> ', end = ' ')
    res_aki = res_aki.drop_duplicates()
    
    print(f'Drop Duplicates : {res_aki.shape[0]}', end = '\n')

    res_aki = pd.merge(res_aki, stays[['stay_id', 'admissiontime']], how = 'left', on = 'stay_id')

    return res_aki[res_aki['datetime'] >= res_aki['admissiontime']][use_cols].reset_index(drop=True)

# Disease Annotation
aki_annot_df = add_aki_annotation(baseline_flag=aki_baseline_flag, urine_flag=aki_urine_flag, stays=ex_icustays_df)

gc.collect()

Drop Duplicates : 179242 -->  Drop Duplicates : 177713


0

In [26]:
# Free up for memory
del baseline_7d_df, baselines_df, urine_rate_df, adm_weights
gc.collect()

0

# 3) Extract Final dataframe

- 해야할 작업 : 같은 charttime의 var이면 평균내기, 각 환자마다 Intime 이후 24시간까지의 값만 활용, demographic 정보까지 통합하기 

In [27]:
# urine rate은 삭제
urine_df = observation_df[observation_df['var_name']=='urine'].reset_index(drop=True)
observation_df = observation_df[~observation_df['var_name'].isin(['urine','urinerate'])].reset_index(drop=True)

In [28]:
def filter_until_24hours(df, stays, type):

    if type == 'output':
        use_cols = ['stay_id', 'datetime', 'var_name', 'value']
    else:
        use_cols = ['stay_id', 'datetime', 'var_name', 'value']

    if 'admissiontime' not in df.columns:
        print('Merging!')
        res = pd.merge(df[use_cols], stays[['stay_id', 'admissiontime']], how = 'left', on = 'stay_id')
    else:
        res = df.copy()
        
    res['max_time'] = res['admissiontime'] + pd.Timedelta(days=1)
    res = res[res['datetime'] <= res['max_time']][use_cols]

    if type == 'output':
        res = res.groupby(['stay_id', 'datetime', 'var_name'],as_index = False)['value'].sum()
        
    else:
        res = res.groupby(['stay_id', 'datetime', 'var_name'],as_index = False)['value'].mean()

    res = res.sort_values(['stay_id', 'datetime'])
    res.rename(columns={'datetime' : 'charttime'}, inplace = True)

    return res.reset_index(drop=True)

filtered_observation_df = filter_until_24hours(observation_df, ex_icustays_df, 'chart')
filtered_outputevents_df = filter_until_24hours(urine_df, ex_icustays_df, 'output')

In [29]:
# static information
static_df = ex_icustays_df[['stay_id', 'age', 'sex']].rename(columns={'sex':'gender'}).melt(
    id_vars = ['stay_id'],
    value_vars = ['age', 'gender'],
    var_name = 'var_name',
    value_name = 'value'
)

# static_df에 weight 추가로 넣기
wh_df = filtered_observation_df[filtered_observation_df['var_name'].isin(['weight', 'height'])].groupby(['stay_id', 'var_name'],as_index = False).mean()[['stay_id', 'var_name', 'value']]

static_df = pd.concat([static_df, wh_df], axis = 0).reset_index(drop=True)

del wh_df
gc.collect()

display(static_df.head())
display(static_df.var_name.unique())

,stay_id,var_name,value
0,2,age,80
1,4,age,75
2,5,age,20
3,7,age,65
4,9,age,70


array(['age', 'gender', 'height', 'weight'], dtype=object)

In [30]:
# Final 합치기 
dynamics_df = pd.concat([filtered_observation_df, filtered_outputevents_df], axis = 0).sort_values(['stay_id','charttime']).reset_index(drop=True)

In [31]:
del filtered_observation_df, filtered_outputevents_df
gc.collect()

0

# 4) Mortality, LoS, Disease Prediction Labeling

In [32]:
# Reference : An Extensive Data Preprocessing Pipeline for MIMIC-IV

# Reference : An Extensive Data Preprocessing Pipeline for MIMIC-IV
def add_mortality_status(x):
    if x == "dead":
        return True
    else:
        return False

def add_inunit_mortality_to_icustays(stays):
    df = stays.copy()
    df['mortality_inunit'] = df['discharge_status'].apply(add_mortality_status).astype(int)
    
    return df

def add_los_to_icustays(stays): 
    """
    각 stay_id마다 admission time과 last observation time을 고려해서 계산해야함.
    """
    df = stays.copy()
    df['los_reg'] = ((df['los'] - 1) * 24).astype(int) 
    df['los_reg'] = df['los_reg'].clip(upper=168)
    
    return df

# 규민이형 말씀 : AKI를 training 할 때는 입원 후 하루 기간동안 AKI가 왔어도 사용한 경우가 있으나, 평가할 때는 빼는 게 맞는 거 같다는 생각.
def prediction_labeling(df : pd.DataFrame, annot_df : pd.DataFrame, prediction_hour : int = 8,
                        label_col_name : str = 'AKI_next_6h', annot_col_name : str = 'AKI_Annotation'):
    
    def _labeling(x):

        labels = [
            1 if x[annot_col_name].iloc[idx+1:idx+1+prediction_window].sum() >= 1 else 0
            for idx in range(len(x))
        ]
        x[label_col_name] = labels

        return x

    df['intime_next_24h'] = df['intime'] + pd.Timedelta(days=1)

    annot_df[[]]
    labeled_df = pd.merge(df, annot_df, how = 'left', on = ['stay_id', 'time_since_ICU'])
    labeled_df[annot_col_name] = labeled_df[annot_col_name].fillna(0)

    # generates sofa_next12h values within each stay, ensuring that predictions don’t span across different stays
    # The sofa_next12h column contains 1 if a future high SOFA score is anticipated and 0 otherwise, providing an actionable label for predictive modeling
    labeled_df = labeled_df.groupby('stay_id', group_keys=False).progress_apply(_labeling)

    return labeled_df

In [34]:
stays = ex_icustays_df.copy()

outcome_df = add_inunit_mortality_to_icustays(stays)
outcome_df = add_los_to_icustays(outcome_df)

# # # Diesease Early Prediction task
# ## 이 AKI를 어떻게 라벨링 할지 코드 다시 짜야함.
# preidiction_hour = 6 # 6h
# outcome_df = prediction_labeling(outcome_df, aki_annot_df, preidiction_hour, 'AKI_next_6h', 'AKI_Annotation') 

In [35]:
# AKI 라벨링 시 Cohort를 다르게 할지 아니면 그대로 할지 이런 것들도 고민해봐야 함.
aki_annot_df

,stay_id,datetime,AKI_Annotation
0,2,2117-10-19 06:00:00,0
1,2,2117-10-20 06:00:00,0
2,4,2149-01-09 06:00:00,0
3,4,2149-01-10 13:48:00,0
4,5,2144-06-07 06:00:00,0
...,...,...,...
177642,33904,2162-03-21 20:42:00,0
177643,33904,2162-03-21 21:15:00,0
177644,33904,2162-03-21 21:30:00,0
177645,33904,2162-03-21 22:07:00,0


In [36]:
dynamics_df.groupby('stay_id')['charttime'].count().describe()

count    13103.000000
mean      4675.519881
std       1431.892051
min          2.000000
25%       3641.000000
50%       4780.000000
75%       5621.000000
max      12225.000000
Name: charttime, dtype: float64

In [37]:
# 저장할 데이터프레임들
static_df
dynamics_df
outcome_df
aki_annot_df

,stay_id,datetime,AKI_Annotation
0,2,2117-10-19 06:00:00,0
1,2,2117-10-20 06:00:00,0
2,4,2149-01-09 06:00:00,0
3,4,2149-01-10 13:48:00,0
4,5,2144-06-07 06:00:00,0
...,...,...,...
177642,33904,2162-03-21 20:42:00,0
177643,33904,2162-03-21 21:15:00,0
177644,33904,2162-03-21 21:30:00,0
177645,33904,2162-03-21 22:07:00,0


# 5) Final DataFrame

In [ ]:
# MIMIC에서는 기존의 charttime 형태 -> eICU처럼 입원 후 몇 분에 대한 timestamp인지를 나타내도록 변경
def charttime_to_min(df, stays):
    res = pd.merge(df, stays[['stay_id', 'admissiontime']], how = 'left', on ='stay_id')

    res['charttime'] = ((res['charttime'] - res['admissiontime']).dt.total_seconds() / 60).apply(np.ceil).astype(int)

    return res.drop(columns = 'admissiontime')

aki_annot_df.rename(columns={'datetime' : 'charttime'},inplace = True)
dynamics_df = charttime_to_min(dynamics_df, ex_icustays_df)
aki_annot_df = charttime_to_min(aki_annot_df, ex_icustays_df)

In [44]:
# dynamics_df에서 hegiht weight는 제거
dynamics_df = dynamics_df[~dynamics_df['var_name'].isin(['weight', 'height'])].sort_values(['stay_id', 'charttime']).reset_index(drop=True)

In [45]:
# Export Data
dynamics_df.to_csv(ROOT_DIR/'preprocessed_yaib'/'dynamics_df.csv.gz', compression='gzip', index=False)
static_df.to_csv(ROOT_DIR/'preprocessed_yaib'/'static_df.csv.gz', compression='gzip', index=False)
outcome_df.to_csv(ROOT_DIR/'preprocessed_yaib'/'outcome_df.csv.gz', compression='gzip', index=False)
aki_annot_df.to_csv(ROOT_DIR/'preprocessed_yaib'/'aki_annot_df.csv.gz', compression='gzip', index=False)
np.save(ROOT_DIR/'preprocessed_yaib'/'aki_exclude_stayid.npy', np.array(aki_excluded_stays))

# 6) Analysis of events

In [3]:
ROOT_DIR = '/Users/korea/EHRTTA/data/hirid'
ROOT_DIR = Path(ROOT_DIR)

# dynamics_df = pd.read_csv(ROOT_DIR/'preprocessed_yaib'/'dynamics_df.csv.gz', compression='gzip')
static_df = pd.read_csv(ROOT_DIR/'static_df.csv.gz', compression='gzip')

# print(dynamics_df.var_name.nunique())
print(static_df.var_name.nunique())


4


In [4]:
# static data의 anchor age, gender 변경
mapping = {'anchor_age' : 'age', 'gender' : 'sex'}
static_df['var_name'] = static_df['var_name'].replace(mapping)

static_df.to_csv(ROOT_DIR/'static_df.csv.gz', compression='gzip', index = False)

In [37]:
lab_list = MAPPING_DF[~MAPPING_DF['category'].isin(['respiratory', 'vitals', 'output', 'demographics'])].var_name.unique()
vital_list = MAPPING_DF[MAPPING_DF['category'].isin(['respiratory', 'vitals'])].var_name.unique()
output_list = MAPPING_DF[MAPPING_DF['category'].isin(['output'])].var_name.unique()

lab = dynamics_df[dynamics_df['var_name'].isin(lab_list)].groupby('stay_id',as_index = False)['charttime'].count().rename(columns={'charttime' : 'lab'})
vitals = dynamics_df[dynamics_df['var_name'].isin(vital_list)].groupby('stay_id',as_index = False)['charttime'].count().rename(columns={'charttime' : 'vitals'})
output = dynamics_df[dynamics_df['var_name'].isin(output_list)].groupby('stay_id',as_index = False)['charttime'].count().rename(columns={'charttime' : 'output'})

analysis_df = pd.merge(lab, vitals, how='left', on ='stay_id').merge(output, how='left', on='stay_id')

analysis_df['total'] = analysis_df['lab'] + analysis_df['vitals'] + analysis_df['output']

for col in analysis_df.columns:
    if col != 'stay_id':
        print(f'----------------Number of {col} events----------------')
        display(analysis_df[col].describe())

----------------Number of lab events----------------


count    13031.000000
mean       438.978896
std        328.001735
min          1.000000
25%         72.000000
50%        481.000000
75%        772.000000
max       1075.000000
Name: lab, dtype: float64

----------------Number of vitals events----------------


count    13020.000000
mean      4239.724194
std       1234.164330
min          1.000000
25%       3526.000000
50%       4178.000000
75%       4852.250000
max      11554.000000
Name: vitals, dtype: float64

----------------Number of output events----------------


count    13011.000000
mean        16.801860
std          8.132857
min          1.000000
25%         12.000000
50%         15.000000
75%         20.000000
max        107.000000
Name: output, dtype: float64

----------------Number of total events----------------


count    13011.000000
mean      4699.063408
std       1399.171613
min         13.000000
25%       3647.000000
50%       4793.000000
75%       5624.000000
max      12223.000000
Name: total, dtype: float64